In [1]:
import os
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"]="python3"

CIK_FILE = "cik.csv"

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Task 3") \
    .getOrCreate()

In [8]:
df = spark.read.csv(CIK_FILE, header=True, inferSchema=True)

In [9]:
df.printSchema()

root
 |-- regin: string (nullable = true)
 |-- tik: string (nullable = true)
 |-- uik: string (nullable = true)
 |-- registered_voters: integer (nullable = true)
 |-- total_ballots: integer (nullable = true)
 |-- early_voters: integer (nullable = true)
 |-- tik_voters: integer (nullable = true)
 |-- home_voters: integer (nullable = true)
 |-- empty_ballots: integer (nullable = true)
 |-- home_ballots: integer (nullable = true)
 |-- tik_ballots: integer (nullable = true)
 |-- bad_ballots: integer (nullable = true)
 |-- normal_ballots: integer (nullable = true)
 |-- missing_ballots: integer (nullable = true)
 |-- extra_ballots: integer (nullable = true)
 |-- baburin: integer (nullable = true)
 |-- grudinin: integer (nullable = true)
 |-- zhirinovsky: integer (nullable = true)
 |-- putin: integer (nullable = true)
 |-- sobchak: integer (nullable = true)
 |-- suraykin: integer (nullable = true)
 |-- titov: integer (nullable = true)
 |-- yavlinsky: integer (nullable = true)



### 1

Найти явку (%) по всем регионам, результат отсортировать по убыванию:

    - посчитать явку на каждом участке
    - сгруппировать по региону и посчитать среднее
    - отсортировать

In [10]:
df.select(df.regin, ((df.normal_ballots + df.bad_ballots) / df.registered_voters * 100).alias("turnout")) \
    .groupBy("regin") \
    .agg(F.avg("turnout")) \
    .orderBy("avg(turnout)", ascending=False) \
    .show(truncate=False)

+-----------------------------------+-----------------+
|regin                              |avg(turnout)     |
+-----------------------------------+-----------------+
|Территория за пределами РФ         |97.81482127324358|
|Республика Тыва                    |95.77720692264757|
|Кабардино-Балкарская Республика    |91.84953717899113|
|Ямало-Ненецкий автономный округ    |91.43492676458001|
|Республика Северная Осетия - Алания|90.80432438557035|
|Чукотский автономный округ         |90.65933237382981|
|Республика Дагестан                |87.94640119808045|
|Карачаево-Черкесская Республика    |87.53169513307593|
|Кемеровская область                |87.29051234314764|
|Республика Татарстан (Татарстан)   |86.9074240855922 |
|Чувашская Республика - Чувашия     |85.98893760812214|
|Тюменская область                  |85.11380213507125|
|Брянская область                   |84.30460466292507|
|Камчатский край                    |82.53176626699467|
|Республика Ингушетия               |82.44954307

### 2

Выбрать произвольного кандидата и найти тот избирательный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

    - отфильтровать участки
    - посчитать результат кандидата
    - отсортировать по убыванию и оставить только один результат

In [11]:
zh_df = df.select(df.regin, df.uik, df.zhirinovsky, (df.normal_ballots + df.bad_ballots).alias("voters"))
zh_df.show(5, truncate=False)

+--------------------------+------+-----------+------+
|regin                     |uik   |zhirinovsky|voters|
+--------------------------+------+-----------+------+
|Республика Адыгея (Адыгея)|УИК №1|32         |2169  |
|Республика Адыгея (Адыгея)|УИК №2|65         |2616  |
|Республика Адыгея (Адыгея)|УИК №3|13         |2739  |
|Республика Адыгея (Адыгея)|УИК №4|12         |1999  |
|Республика Адыгея (Адыгея)|УИК №5|6          |687   |
+--------------------------+------+-----------+------+
only showing top 5 rows



In [12]:
zh_df.filter((zh_df.voters > 300) & (zh_df.voters != 0)) \
    .select(zh_df.regin, zh_df.uik, (zh_df.zhirinovsky / zh_df.voters * 100).alias("votes")) \
    .orderBy("votes", ascending=False) \
    .limit(1) \
    .show(truncate=False)

+--------------------+---------+-----------------+
|regin               |uik      |votes            |
+--------------------+---------+-----------------+
|Новгородская область|УИК №1194|37.91469194312796|
+--------------------+---------+-----------------+



#### 2.1
Наверное разумнее не сортировать, а найти максимальное значение и отфильтровать

In [13]:
zh_df = zh_df \
    .filter((zh_df.voters > 300) & (zh_df.voters != 0)) \
    .select(zh_df.regin, zh_df.uik, (zh_df.zhirinovsky / zh_df.voters * 100).alias("votes"))

max = zh_df \
    .agg(F.max("votes")) \
    .first() \
    .asDict()["max(votes)"]

zh_df.filter(zh_df.votes == max)\
    .show()

+--------------------+---------+-----------------+
|               regin|      uik|            votes|
+--------------------+---------+-----------------+
|Новгородская область|УИК №1194|37.91469194312796|
+--------------------+---------+-----------------+



### 3

Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна (учитывать %)

    - посчитать явку на УИК
    - посчитать среднюю явку на тик сгруппировав по региону + ТИК
    - посчитать разницу для каждого региону сгруппировав по региону
    - отсортировать по убыванию и оставить только один результат

In [14]:
regin_tik_df = df \
    .select(df.regin, df.tik, ((df.normal_ballots + df.bad_ballots) / df.registered_voters * 100).alias("turnout")) \
    .groupBy("regin", "tik") \
    .agg(F.avg("turnout"))

regin_tik_df.show(5, truncate=False)

+-------------------+---------------------------+-----------------+
|regin              |tik                        |avg(turnout)     |
+-------------------+---------------------------+-----------------+
|Республика Алтай   |Чойская                    |68.37717552863222|
|Республика Дагестан|Акушинская                 |89.90036852192821|
|Республика Калмыкия|Яшалтинская                |71.87087350371634|
|Республика Карелия |Петрозаводская городская №2|63.4496183322635 |
|Алтайский край     |Третьяковская              |69.01511122478819|
+-------------------+---------------------------+-----------------+
only showing top 5 rows



In [15]:
regin_tik_df \
    .groupBy("regin") \
    .agg((F.max("avg(turnout)") - F.min("avg(turnout)")).alias("turnout_delta")) \
    .orderBy("turnout_delta", ascending=False) \
    .limit(1) \
    .show(truncate=False)

+---------------------+-----------------+
|regin                |turnout_delta    |
+---------------------+-----------------+
|Архангельская область|50.37901133787615|
+---------------------+-----------------+



\* можно так же как и в предыдущем пункте вместо сортировки найти наибольшее значение и отфильтровать

### 4

Посчитать дисперсию по явке для каждого региона (по УИК)

    - посчитать явку на каждом участке (УИК)
    - сгруппировать по региону и посчитать дисперсию

In [16]:
df.select(df.regin, ((df.normal_ballots + df.bad_ballots) / df.registered_voters * 100).alias("turnout")) \
    .groupBy("regin") \
    .agg(F.variance("turnout")) \
    .show(truncate=False)

+-----------------------------------+------------------+
|regin                              |var_samp(turnout) |
+-----------------------------------+------------------+
|Республика Саха (Якутия)           |129.38672389188005|
|Республика Хакасия                 |137.49163498497006|
|Республика Ингушетия               |48.685178982159314|
|Удмуртская Республика              |93.30475533570238 |
|Камчатский край                    |284.71418481428884|
|Республика Башкортостан            |151.0785318805998 |
|Карачаево-Черкесская Республика    |151.4816570631167 |
|Республика Дагестан                |167.81632339869694|
|Республика Татарстан (Татарстан)   |200.85561042148595|
|Ставропольский край                |179.86003038626646|
|Республика Северная Осетия - Алания|62.877809869548344|
|Хабаровский край                   |190.76018122927016|
|Республика Марий Эл                |67.13099588832908 |
|Республика Адыгея (Адыгея)         |266.77504245812435|
|Республика Бурятия            

### 5

Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

Добавить общее кол-во голосов, чтобы не пересчитывать

In [17]:
cand_df = df \
    .select("*", (df.normal_ballots + df.bad_ballots).alias("voters"))

Получить список кандидатов

In [18]:
candidates = df.columns[-8:]
candidates

['baburin',
 'grudinin',
 'zhirinovsky',
 'putin',
 'sobchak',
 'suraykin',
 'titov',
 'yavlinsky']

Для каждого кандидата:

    - посчитать результат (в % и округлить)
    - сгруппировать значения по этому результату и подсчитать кол-во

In [19]:
for candidate in candidates: 
    print(candidate)
    cand_df \
        .filter(cand_df.voters != 0) \
        .select(cand_df.uik, F.round(cand_df[candidate] / cand_df.voters * 100).alias("votes")) \
        .groupBy("votes") \
        .count() \
        .orderBy("votes", ascending=False) \
        .show(100)

baburin


+-----+-----+
|votes|count|
+-----+-----+
| 44.0|    1|
| 23.0|    1|
| 21.0|    1|
| 20.0|    2|
| 18.0|    2|
| 17.0|    1|
| 14.0|    2|
| 13.0|    1|
| 11.0|    5|
| 10.0|    7|
|  9.0|    9|
|  8.0|   20|
|  7.0|   17|
|  6.0|   31|
|  5.0|   80|
|  4.0|  166|
|  3.0|  518|
|  2.0| 3530|
|  1.0|46182|
|  0.0|42319|
+-----+-----+

grudinin


+-----+-----+
|votes|count|
+-----+-----+
| 80.0|    1|
| 78.0|    1|
| 75.0|    1|
| 71.0|    1|
| 70.0|    1|
| 69.0|    1|
| 68.0|    1|
| 67.0|    3|
| 61.0|    1|
| 60.0|    1|
| 58.0|    3|
| 57.0|    5|
| 56.0|    4|
| 55.0|    1|
| 54.0|    5|
| 53.0|    3|
| 52.0|    5|
| 51.0|    2|
| 50.0|   14|
| 49.0|    4|
| 48.0|   10|
| 47.0|   10|
| 46.0|   15|
| 45.0|   15|
| 44.0|   15|
| 43.0|   17|
| 42.0|   21|
| 41.0|   21|
| 40.0|   31|
| 39.0|   26|
| 38.0|   51|
| 37.0|   43|
| 36.0|   58|
| 35.0|   68|
| 34.0|   70|
| 33.0|  112|
| 32.0|  106|
| 31.0|  129|
| 30.0|  169|
| 29.0|  214|
| 28.0|  296|
| 27.0|  354|
| 26.0|  438|
| 25.0|  569|
| 24.0|  760|
| 23.0|  871|
| 22.0| 1045|
| 21.0| 1266|
| 20.0| 1498|
| 19.0| 1868|
| 18.0| 2317|
| 17.0| 2874|
| 16.0| 3638|
| 15.0| 4730|
| 14.0| 5922|
| 13.0| 7145|
| 12.0| 7534|
| 11.0| 7395|
| 10.0| 6772|
|  9.0| 5913|
|  8.0| 5149|
|  7.0| 4393|
|  6.0| 3864|
|  5.0| 3684|
|  4.0| 3355|
|  3.0| 2884|
|  2.0| 2323|
|  1.0| 1700|
|  0.0

+-----+-----+
|votes|count|
+-----+-----+
|100.0|  237|
| 99.0|  308|
| 98.0|  576|
| 97.0|  684|
| 96.0|  746|
| 95.0| 1067|
| 94.0|  962|
| 93.0| 1138|
| 92.0| 1289|
| 91.0| 1324|
| 90.0| 1667|
| 89.0| 1402|
| 88.0| 1638|
| 87.0| 1760|
| 86.0| 2012|
| 85.0| 2295|
| 84.0| 2262|
| 83.0| 2527|
| 82.0| 2574|
| 81.0| 2905|
| 80.0| 3233|
| 79.0| 3522|
| 78.0| 3998|
| 77.0| 4545|
| 76.0| 5079|
| 75.0| 5556|
| 74.0| 5376|
| 73.0| 5199|
| 72.0| 4655|
| 71.0| 4037|
| 70.0| 3346|
| 69.0| 2806|
| 68.0| 2279|
| 67.0| 1869|
| 66.0| 1450|
| 65.0| 1207|
| 64.0| 1069|
| 63.0|  907|
| 62.0|  682|
| 61.0|  539|
| 60.0|  435|
| 59.0|  326|
| 58.0|  267|
| 57.0|  217|
| 56.0|  153|
| 55.0|  136|
| 54.0|   97|
| 53.0|   80|
| 52.0|   72|
| 51.0|   35|
| 50.0|   57|
| 49.0|   24|
| 48.0|   30|
| 47.0|   29|
| 46.0|   23|
| 45.0|   18|
| 44.0|   15|
| 43.0|   12|
| 42.0|   13|
| 41.0|    6|
| 40.0|   10|
| 39.0|    6|
| 38.0|    8|
| 37.0|    7|
| 36.0|   13|
| 35.0|    7|
| 34.0|    3|
| 33.0|   10|
| 32.0